In [4]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


In [10]:
import cv2
import os

# Create a directory to save the recorded data
os.makedirs('sign_language_dataset', exist_ok=True)

# Function to record video of sign language gestures
def record_gesture_video(gesture_name, num_samples=100):
    cap = cv2.VideoCapture(0)  # Open the webcam
    
    # Create a directory for the gesture
    os.makedirs(os.path.join('recorded_data', gesture_name), exist_ok=True)
    
    # Record video
    print(f"Recording {gesture_name} gestures...")
    for i in range(num_samples):
        ret, frame = cap.read()  # Read a frame from the webcam
        if not ret:
            print("Failed to capture frame")
            break
        
        # Display the frame
        cv2.imshow('Recording Gesture', frame)
        
        # Save the frame as an image
        img_path = os.path.join('sign_language_dataset', gesture_name, f'{gesture_name}_{i}.jpg')
        cv2.imwrite(img_path, frame)
        
        # Press 'q' to stop recording
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    print(f"{num_samples} {gesture_name} gestures recorded!")
    
    # Release the webcam and close the OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

# Record gestures
record_gesture_video('hello', num_samples=100)
record_gesture_video('love', num_samples=100)
record_gesture_video('drink', num_samples=100)
record_gesture_video('eat', num_samples=100)
record_gesture_video('know', num_samples=100)
record_gesture_video('child', num_samples=100)
record_gesture_video('forget', num_samples=100)
# Repeat for other gestures...


Recording hello gestures...
100 hello gestures recorded!
Recording love gestures...
100 love gestures recorded!
Recording drink gestures...
100 drink gestures recorded!
Recording eat gestures...
100 eat gestures recorded!
Recording know gestures...
100 know gestures recorded!
Recording child gestures...
100 child gestures recorded!
Recording forget gestures...
100 forget gestures recorded!


In [12]:
import os
import shutil

# Define paths
recorded_data_dir = 'sign_language_dataset'
train_dir = 'train/dataset'
test_dir = 'test/dataset'

# Create train and test directories
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Move a percentage of data to the test directory
test_split = 0.2  # Adjust as needed
for gesture_folder in os.listdir(recorded_data_dir):
    gesture_path = os.path.join(recorded_data_dir, gesture_folder)
    train_gesture_path = os.path.join(train_dir, gesture_folder)
    test_gesture_path = os.path.join(test_dir, gesture_folder)
    os.makedirs(train_gesture_path, exist_ok=True)
    os.makedirs(test_gesture_path, exist_ok=True)
    
    files = os.listdir(gesture_path)
    num_files_test = int(len(files) * test_split)
    test_files = files[:num_files_test]
    train_files = files[num_files_test:]
    
    for file in test_files:
        shutil.move(os.path.join(gesture_path, file), os.path.join(test_gesture_path, file))
    for file in train_files:
        shutil.move(os.path.join(gesture_path, file), os.path.join(train_gesture_path, file))


In [17]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Output layer with 'num_classes' units
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)                    │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 24, 24, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 12, 12, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 18432)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 512)                 │       9,437,696 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 7)                   │           3,591 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 9,682,119 (36.93 MB)

 Trainable params: 9,682,119 (36.93 MB)

 Non-trainable params: 0 (0.00 B)

In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax')  # Assuming 10 classes (gestures)
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)                   │ (None, 224, 224, 16)        │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_12 (MaxPooling2D)      │ (None, 112, 112, 16)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_13 (Conv2D)                   │ (None, 112, 112, 32)        │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_13 (MaxPooling2D)      │ (None, 56, 56, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_14 (Conv2D)                   │ (None, 56, 56, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_14 (MaxPooling2D)      │ (None, 28, 28, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 50176)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 512)                 │      25,690,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 25,719,338 (98.11 MB)

 Trainable params: 25,719,338 (98.11 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
# Define the number of classes in your dataset
num_classes = 7  # Update this with the actual number of classes in your dataset

# Define the input shape based on your data
input_shape = (224, 224, 3)  # Update this with the actual shape of your input data

# Define the model architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])


In [38]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Evaluate the model using the validation data generator
evaluation = model.evaluate(validation_generator)

# Extract and print the loss and accuracy
loss = evaluation[0]
accuracy = evaluation[1]
print("Validation Loss: ", loss)
print("Validation Accuracy: ", accuracy)


ValueError: Must provide at least one structure